In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import pandas as pd

import os

In [11]:
# data = pd.read_csv('./data/processedData/12-07-18-08.csv')
# path =r'C:\DRO\DCL_rawdata_files' # use your path
path = os.path.expanduser("~/multiPartyHRI/data/labeledData/")
allFiles = os.path.join(path + "*.csv")
# frame = pd.DataFrame()
# list_ = []
# for file_ in allFiles:
#     df = pd.read_csv(file_,index_col=None, header=0)
#     list_.append(df)
# frame = pd.concat(list_)

df = pd.concat((pd.read_csv(f, engine='python') for f in allFiles))

IsADirectoryError: [Errno 21] Is a directory: '/'

In [7]:
allFiles

'/home/sp/multiPartyHRI/data/labeledData/*.csv'

In [15]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
#     "body_acc_y_",
#     "body_acc_z_",
#     "body_gyro_x_",
#     "body_gyro_y_",
#     "body_gyro_z_",
#     "total_acc_x_",
#     "total_acc_y_",
#     "total_acc_z_"
]
# INPUT_SIGNAL_TYPES = list(data.columns)

# Output classes to learn how to classify
LABELS = [
    "WALKING", 
#     "WALKING_UPSTAIRS", 
#     "WALKING_DOWNSTAIRS", 
#     "SITTING", 
#     "STANDING", 
#     "LAYING"
#     "Approaching",
#     "Interacting",
#     "Leaving",
#     "Uninterested"
] 

In [16]:
DATA_PATH = "data/"
DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
# print("\n" + "Dataset is now located at: " + DATASET_PATH)

In [17]:
TRAIN = "train/"
TEST = "test/"

In [18]:
# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()
    
    return np.transpose(np.array(X_signals), (1, 2, 0))

In [19]:
X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)

In [21]:
X_train.shape

(2947, 128, 1)

In [22]:
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    # Substract 1 to each output class for friendly 0-based indexing 
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

In [24]:
y_test.shape

(2947, 1)